# Streaming

In [1]:
import findspark
findspark.init("/home/bryan/Documents/Code/spark-2.4.5-bin-hadoop2.7")

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('stream').getOrCreate()

In [3]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [4]:
from pyspark.sql.functions import explode, split

In [5]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

## Open a new console, start netcat with this command: 'nc -lk 9999'

> ### Anything typed into that terminal will be "heard" by the Spark streaming session 

In [6]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

KeyboardInterrupt: 

# END